# AutoGen eszköz használati példa


## Importálja a szükséges csomagokat


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Az ügyfél létrehozása

Ebben a példában a [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) szolgáltatást fogjuk használni az LLM eléréséhez.

A `model` értéke `gpt-4o-mini`. Próbáld ki, hogy megváltoztatod a modellt egy másikra, amely elérhető a GitHub Models piacterén, és nézd meg a különböző eredményeket.

Gyors tesztként csak egy egyszerű kérdést futtatunk: `Mi Franciaország fővárosa`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## A funkciók meghatározása

Ebben a példában az ügynök hozzáférést kap egy eszközhöz, amely egy funkció, amely elérhető nyaralási célpontok listáját és azok elérhetőségét tartalmazza.

Ezt úgy képzelheted el, mint egy olyan helyzetet, ahol egy utazási ügynök például hozzáférhet egy utazási adatbázishoz.

Ahogy végigmész ezen a példán, bátran próbálj meg új funkciókat és eszközöket definiálni, amelyeket az ügynök használhat.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## A funkcióeszköz meghatározása  
Ahhoz, hogy az ügynök a `vacation_destinations`-t `FunctionTool`-ként használja, először definiálnunk kell azt.  

Ezenkívül megadunk egy leírást az eszközről, amely segít az ügynöknek azonosítani, hogy az eszköz mire használható a felhasználó által kért feladat kapcsán.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Az ügynök meghatározása

Most létrehozhatjuk az ügynököt az alábbi kódban. A `system_message` segítségével adjuk meg az ügynöknek az utasításokat, hogy miként segítse a felhasználókat nyaralási úti célok megtalálásában.

A `reflect_on_tool_use` paramétert is igazra állítjuk. Ez lehetővé teszi, hogy az LLM a segédeszköz hívásának válaszát feldolgozza, és természetes nyelven küldje el a választ.

A paramétert hamisra is állíthatod, hogy lásd a különbséget.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Az ügynök futtatása

Most futtathatjuk az ügynököt azzal a kezdeti üzenettel, amelyben a felhasználó egy tokiói utazást kér. 

Megváltoztathatod ezt a város célállomást, hogy megnézd, hogyan reagál az ügynök a város elérhetőségére.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
